In [17]:

import sys
import os
import numpy as np


In [2]:
LIB = os.getcwd().replace('datasets/ToxValDB','')


In [3]:
if not LIB in sys.path:
    sys.path.insert(0,LIB)

In [4]:
from logger import setup_applevel_logger

In [5]:
log = setup_applevel_logger(file_name ='logs/prepare_dataset_ToxVal_sdf.log')

from pathlib import Path
import pandas as pd
from rdkit import Chem
import json
from collections import Counter
from tqdm import tqdm
import openpyxl

from cheminformatics.rdkit_toolkit import convert_smiles_to_mol, Rdkit_operation
from rdkit.Chem.MolStandardize import rdMolStandardize


2024-07-12 10:30:39,392 - GNN_muta.cheminformatics.rdkit_toolkit - INFO - Converting smiles to rdkit molecules


In [6]:

# pandas display options
# do not fold dataframes
pd.set_option('expand_frame_repr',False)
# maximum number of columns
pd.set_option("display.max_columns",50)
# maximum number of rows
pd.set_option("display.max_rows",500)
# precision of float numbers
pd.set_option("display.precision",3)
# maximum column width
pd.set_option("max_colwidth", 250)

# enable pandas copy-on-write
pd.options.mode.copy_on_write = True


In [23]:

# read the ToxVal dataset,as extracted from ToxVal9.5_2024_07_02
inpf = Path(r'genetox_data_ToxVal_110724_v1.xlsx')
tox_data = pd.read_excel(inpf, index_col = [0])



In [33]:
tox_data = (tox_data
  .assign( result=lambda x: np.where(x.assay_result == 'positive', 1, 
                     np.where(x.assay_result == 'negative', 0, np.nan)))
          .dropna(subset = ['Structure_SMILES'], axis = 0)
         )

In [ ]:
datapoint_ID = 0
assay_results = []
errors = []
for index, row in tox_data.iterrows():
    assay_result = {'datapoint ID': datapoint_ID}
    assay_result['assay'] = row['assay_type_standard']
    assay_result['endpoint'] = row['assay_type_standard']
    assay_result['parsing notes'] = []
    smiles = row['smiles'] = row['Structure_SMILES']
    mol, error_warning = convert_smiles_to_mol(smiles, sanitize=False)
    if mol is not None:
        try:
            mol = rdMolStandardize.Cleanup(mol)  # default rdKit cleanup
        except Exception as error:
            errors.append(error)
            
            assay_result['parsing notes'].append('parsing initial SMILES: ' + error_warning)
    else:
        log.info(f'could not read molecule {index} from {inpf}')

    # standardise and compute the InChi and InChiKey
    if mol:
        with Rdkit_operation() as sio:
            smiles = Chem.MolToSmiles(mol).strip()  # canonical smiles
            error_warning = sio.getvalue()
        if error_warning:
            assay_result['parsing notes'].append('parsing molecule in rdKit: ' + error_warning)
        if pd.isnull(smiles) or len(smiles) == 0:
            smiles = None
            assay_result['parsing notes'].append('empty smiles')
        assay_result['smiles (canonical)'] = smiles

    # fetch the DTXSID number
    assay_result['dtxsid'] = row['dtxsid']

    # genotoxicity outcome
    genotoxicity = row['assay_result']
    assay_result['genotoxicity'] = genotoxicity
    if pd.isnull(genotoxicity) or genotoxicity not in [0, 1]:
        log.warning(f'no genotoxicity data in row {index}')
        assay_result['parsing notes'].append('no genotoxicity outcome')

    # append the datapoint increment counter
    assay_result['parsing notes'] = '; '.join(assay_result['parsing notes'])
    assay_results.append(assay_result)
    datapoint_ID += 1
assay_results = pd.DataFrame(assay_results).reset_index(drop=True)
assay_results['parsing notes'] = assay_results['parsing notes'].replace('', None)

tmp = (assay_results.groupby(['smiles (canonical)', 'assay'])['genotoxicity'].nunique()>1).rename('conflicting outcomes (assay)').reset_index()
assay_results = assay_results.merge(tmp, on=['smiles (canonical)', 'assay'], how='left')
log.info(f'assay/structure combinations with multiple calls: {assay_results["conflicting outcomes (assay)"].sum()}')
tmp = (assay_results.groupby(['smiles (canonical)', 'endpoint'])['genotoxicity'].nunique()>1).rename('conflicting outcomes (endpoint)').reset_index()
assay_results = assay_results.merge(tmp, on=['smiles (canonical)', 'endpoint'], how='left')
log.info(f'endpoint/structure combinations with multiple calls: {assay_results["conflicting outcomes (endpoint)"].sum()}')
# set the molecule ID
tmp = assay_results['smiles (canonical)'].dropna().drop_duplicates().reset_index().rename({'index': 'molecule ID'}, axis='columns').fillna('-').astype(str)
assay_results = assay_results.merge(tmp, on='smiles (canonical)', how='left')


2024-07-12 10:23:42,528 - GNN_muta.cheminformatics.rdkit_toolkit - INFO - successfully converted smiles [O-][N+](=O)C1=CC=C(Cl)C=C1 to mol
2024-07-12 10:23:42,530 - GNN_muta - WARNING - no genotoxicity data in row 0
2024-07-12 10:23:42,532 - GNN_muta.cheminformatics.rdkit_toolkit - INFO - successfully converted smiles [O-][N+](=O)C1=CC=C(Cl)C=C1 to mol
2024-07-12 10:23:42,534 - GNN_muta - WARNING - no genotoxicity data in row 1
2024-07-12 10:23:42,535 - GNN_muta.cheminformatics.rdkit_toolkit - INFO - successfully converted smiles [O-][N+](=O)C1=CC=C(Cl)C=C1 to mol
2024-07-12 10:23:42,537 - GNN_muta - WARNING - no genotoxicity data in row 2
2024-07-12 10:23:42,539 - GNN_muta.cheminformatics.rdkit_toolkit - INFO - successfully converted smiles [O-][N+](=O)C1=CC=C(Cl)C=C1 to mol
2024-07-12 10:23:42,540 - GNN_muta - WARNING - no genotoxicity data in row 3
2024-07-12 10:23:42,542 - GNN_muta.cheminformatics.rdkit_toolkit - INFO - successfully converted smiles [O-][N+](=O)C1=CC=C(Cl)C=C1 to m

In [34]:
len(assay_results)

42475

In [48]:
assay_results.to_csv('assay_results_120724.csv')

In [43]:
outf ='/home/grace/Documents/python/GNN/datasets/ToxValDB/processed/tabular'

In [45]:
log.info('stored assay results in ' + str(outf + 'ToxValDB_genotoxicity.xlsx'))

2024-07-12 11:45:40,578 - GNN_muta - INFO - stored assay results in /home/grace/Documents/python/GNN/datasets/ToxValDB/processed/tabularToxValDB_genotoxicity.xlsx


In [46]:
pwd

'/home/grace/Documents/python/GNN/datasets/ToxValDB'

In [47]:
with pd.ExcelWriter(outf + 'ToxValDB_genotoxicity.xlsx', engine='openpyxl') as writer:
    # detailed
    assay_results.to_excel(writer, sheet_name='genotoxicity detailed')
    # summary (assay level), this eliminates records with null smiles (canonical)
    tmp_cas = assay_results.groupby('smiles (canonical)')['dtxsid'].unique().rename('dtxsid').apply(lambda x: '; '.join(sorted(x))).reset_index()
    tmp = assay_results.copy()
    tmp['genotoxicity'] = assay_results['genotoxicity'].replace({1.0: 'positive', 0.0: 'negative'})
    tmp = tmp[['molecule ID', 'smiles (canonical)', 'endpoint', 'assay', 'genotoxicity']].merge(tmp_cas, on='smiles (canonical)', how='left')
    tmp = tmp.groupby(['molecule ID', 'smiles (canonical)', 'dtxsid', 'endpoint', 'assay'], dropna=False)['genotoxicity'].agg(lambda x: x.dropna().drop_duplicates().to_list()).apply(lambda x: x[0] if len(x)==1 else 'ambiguous' if len(x)>1 else 'not available').reset_index()
    tmp = tmp.pivot(index=['molecule ID', 'smiles (canonical)', 'dtxsid'], columns=['endpoint', 'assay'], values='genotoxicity').fillna('not available').reset_index()
    tmp = tmp.loc[tmp['smiles (canonical)'].notnull()]
    tmp.to_excel(writer, sheet_name='genotoxicity summary (assay)')
    assay_results_assay = tmp.drop(('dtxsid',''), axis='columns')
    cols = [col for col in assay_results_assay.columns if col != ('molecule ID', '') and  col != ('smiles (canonical)', '') ]
    assay_results_assay = assay_results_assay.melt(id_vars=[('molecule ID', ''), ('smiles (canonical)','')], value_vars=cols, value_name='genotoxicity').rename({('smiles (canonical)',''): 'smiles (canonical)', ('molecule ID', ''): 'molecule ID'}, axis='columns').rename({('smiles (canonical)',''):'smiles (canonical)'}, axis='columns')

    # summary (endpoint level), this eliminates records with null smiles (canonical)
    tmp_cas = assay_results.groupby('smiles (canonical)')['dtxsid'].unique().rename('dtxsid').apply(lambda x: '; '.join(sorted(x))).reset_index()
    tmp = assay_results.copy()
    tmp['genotoxicity'] = assay_results['genotoxicity'].replace({1.0: 'positive', 0.0: 'negative'})
    tmp = tmp[['molecule ID', 'smiles (canonical)', 'endpoint', 'genotoxicity']].merge(tmp_cas, on='smiles (canonical)', how='left')
    tmp = tmp.groupby(['molecule ID', 'smiles (canonical)', 'dtxsid', 'endpoint'], dropna=False)['genotoxicity'].agg(lambda x: x.dropna().drop_duplicates().to_list()).apply(lambda x: x[0] if len(x)==1 else 'ambiguous' if len(x)>1 else 'not available').reset_index()
    tmp = tmp.pivot(index=['molecule ID', 'smiles (canonical)', 'dtxsid'], columns='endpoint', values='genotoxicity').fillna('not available').reset_index()
    tmp = tmp.loc[tmp['smiles (canonical)'].notnull()]
    tmp.to_excel(writer, sheet_name='genotoxicity summary (endpoint)')
    assay_results_endpoint = tmp.drop('dtxsid', axis='columns').melt(id_vars=['molecule ID', 'smiles (canonical)'], value_name='genotoxicity')


In [49]:
outf ='/home/grace/Documents/python/GNN/datasets/ToxValDB/processed/sdf/'

In [54]:
assay_results_assay

,molecule ID,smiles (canonical),endpoint,assay,genotoxicity
0,0,O=[N+]([O-])c1ccc(Cl)cc1,bacterial reverse mutation test,bacterial reverse mutation test,ambiguous
1,10004,[Na+].[O-]c1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl,bacterial reverse mutation test,bacterial reverse mutation test,not available
2,10005,CC1C(C)(C)C2=C(c3ncncc3CC2)C1(C)C,bacterial reverse mutation test,bacterial reverse mutation test,negative
3,10010,COc1ccc(C(=O)c2ccccc2O)c(O)c1,bacterial reverse mutation test,bacterial reverse mutation test,ambiguous
4,10013,COc1ccc(C(=O)c2ccc(OC)cc2O)c(O)c1,bacterial reverse mutation test,bacterial reverse mutation test,not available
...,...,...,...,...,...
173509,9982,CCN(CCO)N=O,DNA damage (in vivo),DNA damage (in vivo),not available
173510,9983,[P-3].[P-3].[Zn+2].[Zn+2].[Zn+2],DNA damage (in vivo),DNA damage (in vivo),not available
173511,9987,s1p2sp3p1p3s2,DNA damage (in vivo),DNA damage (in vivo),not available
173512,9989,[Pb+2].[S-2],DNA damage (in vivo),DNA damage (in vivo),not available


In [59]:
unique_smiles = assay_results['smiles (canonical)'].dropna().unique()

log.info('writing molecules to ' + str(outf + 'ToxValDB_genotoxicity.sdf'))
with Chem.SDWriter(outf + 'ToxValDB_genotoxicity.sdf') as sdf_writer:
    for smiles in tqdm(unique_smiles):
        genotoxicity_outcome_assay = assay_results_assay.loc[assay_results_assay['smiles (canonical)']==smiles].drop('smiles (canonical)', axis='columns').to_json(orient='records')
        genotoxicity_outcome_endpoint = assay_results_endpoint.loc[assay_results_endpoint['smiles (canonical)'] == smiles].drop('smiles (canonical)', axis='columns').to_json(orient='records')
        mol = Chem.MolFromSmiles(smiles)
        mol.SetProp('genotoxicity_assay_level', genotoxicity_outcome_assay)
        mol.SetProp('genotoxicity_endpoint_level', genotoxicity_outcome_endpoint)
        sdf_writer.write(mol)


2024-07-12 12:03:46,012 - GNN_muta - INFO - writing molecules to /home/grace/Documents/python/GNN/datasets/ToxValDB/processed/sdf/ToxValDB_genotoxicity.sdf


AttributeError: 'NoneType' object has no attribute 'SetProp'